<a href="https://colab.research.google.com/github/prakul/MongoDB-AI-Resources/blob/main/llamaIndex%2BmongoDB_MetadataFiltering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pymongo
!pip install llama_index==0.9.17 pypdf
!pip3 install python-dotenv
!pip install openai

In [2]:
import os
from dotenv import load_dotenv
load_dotenv(override=True)

OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
MONGO_URI = os.environ["MONGO_URI"]


In [ ]:
# OPTIONAL - try this if llamaindex throws an OpenAI Authentication error
#import openai
#openai.api_key = OPENAI_API_KEY


In [3]:
import pymongo
from llama_index.vector_stores.mongodb import MongoDBAtlasVectorSearch
from llama_index.indices.vector_store.base import VectorStoreIndex
from llama_index.storage.storage_context import StorageContext
from llama_index.readers.file.base import SimpleDirectoryReader


In [36]:
!mkdir -p 'data/10k/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/10k/uber_2021.pdf' -O 'data/10k/uber_2021.pdf'


--2024-02-16 08:53:41--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/10k/uber_2021.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1880483 (1.8M) [application/octet-stream]
Saving to: ‘data/10k/uber_2021.pdf’

data/10k/uber_2021. 100%[===================>]   1.79M  --.-KB/s    in 0.04s   

2024-02-16 08:53:41 (48.8 MB/s) - ‘data/10k/uber_2021.pdf’ saved [1880483/1880483]



In [38]:

# Load the PDF into LlamaIndex docs format
uber_docs = SimpleDirectoryReader(input_files=["./data/10k/uber_2021.pdf"]).load_data()


In [21]:
print((uber_docs[0].__dict__))

{'id_': '32a146f4-1bf7-42d7-9a54-ed583993a996', 'embedding': None, 'metadata': {'page_label': '1', 'file_name': 'uber_2021.pdf', 'file_path': 'uber_2021.pdf', 'file_type': 'application/pdf', 'file_size': 4557617, 'creation_date': '2024-02-16', 'last_modified_date': '2024-02-16', 'last_accessed_date': '2024-02-16'}, 'excluded_embed_metadata_keys': ['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], 'excluded_llm_metadata_keys': ['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], 'relationships': {}, 'hash': '13c1b36110971e4dfd25578b6fe1576129cd93785e79613c1f602a96c0c68f08', 'text': 'Uber \n2021 \nAnnual Report', 'start_char_idx': None, 'end_char_idx': None, 'text_template': '{metadata_str}\n\n{content}', 'metadata_template': '{key}: {value}', 'metadata_seperator': '\n'}


In [39]:
# Setup the configurations for MongoDBAtlasVectorSearch
# This class is defined at https://github.com/jerryjliu/llama_index/blob/main/llama_index/vector_stores/mongodb.py

MONGODB_DB_NAME = "LlamaIndex_sample" # default is "default_db"
MONGODB_COLLECTION_NAME = "uber_sample_1" # default is "default_collection"
ATLAS_VECTOR_SEARCH_INDEX_NAME = "LlamaIndex_vector_index_1" # default is "default"

In [40]:
mongodb_client = pymongo.MongoClient(MONGO_URI)
#mongodb_client.list_database_names()

In [41]:
from llama_index.vector_stores.mongodb import MongoDBAtlasVectorSearch

store = MongoDBAtlasVectorSearch(mongodb_client, db_name=MONGODB_DB_NAME, collection_name=MONGODB_COLLECTION_NAME, index_name=ATLAS_VECTOR_SEARCH_INDEX_NAME)
storage_context = StorageContext.from_defaults(vector_store=store)


In [42]:
index = VectorStoreIndex.from_documents(uber_docs, storage_context=storage_context, show_progress=True)


Parsing nodes:   0%|          | 0/307 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/410 [00:00<?, ?it/s]

The above steps will result in the LlamaIndex docs stored in MongoDB

# ![picture](https://drive.google.com/uc?id=19k9gv-uoGfV7NngYPi4gk2a64l-GNCRu)

# Create an Index in Atlas Vector Search


Now go to the Search tab on Atlas to create a vector search index on your cluster.  Please refer to the [documentation](https://www.mongodb.com/docs/atlas/atlas-search/define-field-mappings-for-vector-search), [sample blog](https://www.mongodb.com/developer/products/atlas/building-generative-ai-applications-vector-search-open-source-models/#semantic-search-for-movie-recommendations) to get more details on how to define an Atlas Vector Search index.

Name the index `LlamaIndex_vector_index` (as defined in ATLAS_VECTOR_SEARCH_INDEX_NAME), and create the index on the namespace `LlamaIndex_sample.uber_sample`. Finally, write the following definition in the JSON editor on MongoDB Atlas:

```
{
  "fields": [
    {
      "type": "vector",
      "path": "embedding",
      "numDimensions": 1536,
      "similarity": "cosine"
    },
    {
      "type": "filter",
      "path": "metadata.page_label"
    }
  ]
}

```
In the example, `embedding` is the name of the field that contains the embedding vector.


In [47]:
response = index.as_query_engine().query("What was Uber's revenue?", )
print(response)


Uber's revenue was $17,455 million.


In [48]:
print(response.source_nodes)

[NodeWithScore(node=TextNode(id_='fc2b9b5b-ae48-4b9c-a5e1-85ce0af33b52', embedding=None, metadata={'page_label': '129', 'file_name': 'uber_2021.pdf', 'file_path': 'data/10k/uber_2021.pdf', 'file_type': 'application/pdf', 'file_size': 1880483, 'creation_date': '2024-02-16', 'last_modified_date': '2024-02-16', 'last_accessed_date': '2024-02-16'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='4df95547-bb0b-4602-b545-60e63f0ba785', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '129', 'file_name': 'uber_2021.pdf', 'file_path': 'data/10k/uber_2021.pdf', 'file_type': 'application/pdf', 'file_size': 1880483, 'creation_date': '2024-02-16', 'last_modified_date': '2024-02-16', 'last_accessed_date': '

## Metadata Filtering

Each LlamaIndex document contains a metadata field.
The default metadata field contains following fields

```
metadata={'page_label': '129', 'file_name': 'uber_2021.pdf', 'file_path': 'data/10k/uber_2021.pdf', 'file_type': 'application/pdf', 'file_size': 1880483, 'creation_date': '2024-02-16', 'last_modified_date': '2024-02-16', 'last_accessed_date': '2024-02-16'}
```


The metadata can be customized using instructions at
https://docs.llamaindex.ai/en/stable/module_guides/loading/documents_and_nodes/usage_documents.html


Whatever metadata field is desired to be searchable, needs to be included in the Atlas Vector Search index definition. In the example above we have included the field `metadata.page_label` in the Vector Search index definition via the `type: filter`
```
{
      "type": "filter",
      "path": "metadata.page_label"
    }
```

In the above response, we can see that the answer was contained in the page_label = 129.

If we exclude that particular page_label we wont be able to find the corresponding answer

In [55]:
from llama_index.vector_stores import ExactMatchFilter, MetadataFilters

filters = MetadataFilters(
    filters=[ExactMatchFilter(key="metadata.page_label", value="131")]
)
response = index.as_query_engine(filters=filters).query("What was Uber's revenue?", )
print(response)

I'm sorry, but the given context does not provide any information about Uber's revenue.
